![attention_intent_slot-gate模型](img/model.png)


在《Attention-Based Recurrent Neural Network Models for Joint Intent Detection and Slot Filling》中的模型attention-based rNN model基础上，提出了slot-gate门。
    
   通过slot-gate来加强intent与slot任务的交互性。见文章《Slot-Gated Modeling for Joint Slot Filling and Intent Prediction》。
    
   模型步骤：
    
   1.意图识别是利用encoder中的最后一个time step中的双向隐层，利用attention加权平均，最后接一个fc层进行分类
    
   2.槽填充是序列标注，双向隐状态加attention权重，最后接一个fc层分类。
    
     a.上图a中模型结构，利用了slot attention与intent attention。经过gate门后的值与每个时间步中的slot attention进行交互。
        
     b.上图b中模型结构，只利用了intent attention。经过gate门后的值与每个时间步的隐状态进行交互。
        
   3.总的loss = 意图识别loss + 槽填充loss


## attention-based rnn 模型步骤：
    
   1.底层是bilstm或bigru，输入为用户语句序列，输出为隐状态
    
   2.槽填充为序列标注任务，将用户语句序列映射到槽标签中。
        
   ### slot context vector:
   
   槽注意力向量是隐状态加权和：
   
   ![slot-1模型](img/slot_1.png)      (1)
    
   槽注意力权重：
   
   ![slot-2模型](img/slot_2.png)      (2)
   
   最后将隐状态和槽上下文向量$c^S_i$用于预测标签序列：
   
   ![slot-3模型](img/slot_3.png)      (3)
   
   这里的slot context vector就是对于每个位置$i$，有一个对应的前馈网络权重，经过前馈网络和激活函数得到$e_i^S$，经过softmax得到$\alpha^S_i$。再由(1)得到slot上下文向量。
   
   
   3.意图识别为分类任务，用bilstm或bigru最后一个时间步隐状态进行预测。
    
   ### intent context vector:
        
   注意力向量的计算和槽注意力向量计算一致：
   
   ![intent-1模型](img/intent_1.png)      (4)
   
   注意力权重：
   
   ![intent-2模型](img/intent_2.png)      (5)
   
   最后一个隐状态和意图上下文向量$c^I$用于预测意图类别：
   
   ![intent-3模型](img/intent_3.png)      (6)
   
   ### slot-gate机制
   
   ![slot-gate模型](img/gate.png)         (7)
   
   #### 带slot-attention和intent-attention的gate机制
   
   slot-gate的计算：
   
   ![slot-gate1模型](img/gate_1.png)      (8)
   
   这里的$g$可看做是一个加权特征，那么(3)式中的槽序列预测公式可改为：
   
   ![slot-gate2模型](img/gate_2.png)      (9)
   
    g越大，表示slot context vector和intent context vector关注的是输入序列的同一部分，也说明槽与意图之间的相关性更强，则context vector对预测结果的贡献更可靠。
    
   #### 只带intent-attention的gate机制，将(8)和(9)改为如下：
   
   ![slot-gate3模型](img/gate_3.png)      (10)
   

In [1]:
import os
from torchtext import data, datasets
import pandas as pd
import pickle

In [2]:
base_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
atis_data = os.path.join(base_dir, 'atis')

In [3]:
'''
build train and val dataset
'''
    
tokenize = lambda s:s.split()

SOURCE = data.Field(sequential=True, tokenize=tokenize,
                    lower=True, use_vocab=True,
                    init_token='<sos>', eos_token='<eos>',
                    pad_token='<pad>', unk_token='<unk>',
                    batch_first=True, fix_length=50,
                    include_lengths=True) #include_lengths=True为方便之后使用torch的pack_padded_sequence

TARGET = data.Field(sequential=True, tokenize=tokenize,
                    lower=True, use_vocab=True,
                    init_token='<sos>', eos_token='<eos>',
                    pad_token='<pad>', unk_token='<unk>',
                    batch_first=True, fix_length=50,
                    include_lengths=True) #include_lengths=True为方便之后使用torch的pack_padded_sequence
LABEL = data.Field(
                sequential=False,
                use_vocab=True)

train, val = data.TabularDataset.splits(
                                        path=atis_data,
                                        skip_header=True,
                                        train='atis.train.csv',
                                        validation='atis.test.csv',
                                        format='csv',
                                        fields=[('index', None), ('intent', LABEL), ('source', SOURCE), ('target', TARGET)])

SOURCE.build_vocab(train, val)
TARGET.build_vocab(train, val)
LABEL.build_vocab(train, val)

train_iter, val_iter = data.Iterator.splits(
                                            (train, val),
                                            batch_sizes=(32, len(val)), # 训练集设置为64,验证集整个集合用于测试
                                            shuffle=True,
                                            sort_within_batch=True, #为true则一个batch内的数据会按sort_key规则降序排序
                                            sort_key=lambda x: len(x.source)) #这里按src的长度降序排序，主要是为后面pack,pad操作)


In [4]:
SOURCE.vocab.stoi[SOURCE.pad_token]

1

In [5]:
# save source words
source_words_path = os.path.join(os.getcwd(), 'source_words.pkl')
with open(source_words_path, 'wb') as f_source_words:
    pickle.dump(SOURCE.vocab, f_source_words)

# save target words
target_words_path = os.path.join(os.getcwd(), 'target_words.pkl')
with open(target_words_path, 'wb') as f_target_words:
    pickle.dump(TARGET.vocab, f_target_words)
    
# save label words
label_words_path = os.path.join(os.getcwd(), 'label_words.pkl')
with open(label_words_path, 'wb') as f_label_words:
    pickle.dump(LABEL.vocab, f_label_words)

In [6]:
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import math
from apex import amp
import time


In [11]:
# build model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
      
        
# 构建slotgate计算方式，利用slot context与intent context
class SlotGate(nn.Module):
    def __init__(self, hidden_dim):
        super(SlotGate, self).__init__()
        self.fc_intent_context = nn.Linear(hidden_dim, hidden_dim)
        self.fc_v = nn.Linear(hidden_dim, hidden_dim)
        
    def forward(self, slot_context, intent_context):
        '''
        注意这里slot_context是slot上下文context，[batch_size, hidden_dim]，或者是时间步的hidden
        intent_context:[batch_size, hidden_dim]
        '''
        # intent_context_linear:[batch_size, hidden_dim]
        intent_context_linear = self.fc_intent_context(intent_context)
        
        # sum_intent_slot_context:[batch_size, hidden_dim]
        sum_intent_slot_context = slot_context + intent_context_linear
        
        # fc_linear:[batch_size, hidden_dim]
        fc_linear = self.fc_v(torch.tanh(sum_intent_slot_context))
        
        # sum_gate_vec:[batch_size]
        sum_gate_vec = torch.sum(fc_linear, dim=1)
        
        return sum_gate_vec
    
# 这里计算slot context与intent context。就是bigru每个时间步隐藏特征的加权向量，这里不同于原论文，这里使用点乘来计算注意力权重weight
class AttnContext(nn.Module):
    def __init__(self, hidden_dim):
        super(AttnContext, self).__init__()

    def forward(self, hidden, source_output_hidden):
        # source_output_hidden:[batch_size, seq_len, hidden_size]
        # hidden:[batch_size, hidden_size]
        hidden = hidden.unsqueeze(1) # [batch_size, 1, hidden_size]
        
        attn_weight = torch.sum(hidden * source_output_hidden,dim=2) # [batch_size, seq_len]
        
        attn_weight = F.softmax(attn_weight, dim=1).unsqueeze(1) # [batch_size, 1, seq_len]
        
        # 类似于注意力向量
        attn_vector = attn_weight.bmm(source_output_hidden) # [batch_size, 1, hidden_size]
        
        return attn_vector.squeeze(1) # [batch_size, hidden_size]


#构建模型
class BirnnAttentionGate(nn.Module):
    def __init__(self, source_input_dim, source_emb_dim, hidden_dim, n_layers, dropout, pad_index, slot_output_size, intent_output_size, seq_len, predict_flag, slot_attention_flag):
        super(BirnnAttentionGate, self).__init__()
        self.pad_index = pad_index
        self.hidden_dim = hidden_dim//2 # 双向lstm
        self.n_layers = n_layers
        self.slot_output_size = slot_output_size
        # 是否预测模式
        self.predict_flag = predict_flag
        # 原论文中有两种模型结构，一个带slot_attention，一个不带slot_attention
        self.slot_attention_flag = slot_attention_flag
        
        self.source_embedding = nn.Embedding(source_input_dim, source_emb_dim, padding_idx=pad_index)
        # 双向gru，隐层维度是hidden_dim
        self.source_gru = nn.GRU(source_emb_dim, self.hidden_dim, n_layers, dropout=dropout, bidirectional=True, batch_first=True) #使用双向
        
        # slot context
        self.slot_context = AttnContext(hidden_dim)
        
        # intent context
        self.intent_context = AttnContext(hidden_dim)
        
        # slotgate类
        self.slotGate = SlotGate(hidden_dim)
        
        # 意图intent预测
        self.intent_output = nn.Linear(hidden_dim, intent_output_size)
        
        # 槽slot预测
        self.slot_output = nn.Linear(hidden_dim, slot_output_size)
        
        
    def forward(self, source_input, source_len):
        '''
        source_input:[batch_size, seq_len]
        source_len:[batch_size]
        '''
        if self.predict_flag:
            assert len(source_input) == 1, '预测时一次输入一句话'
            seq_len = source_len[0]
            
            # 将输入的source进行编码
            # source_embedded:[batch_size, seq_len, source_emb_dim]
            source_embedded = self.source_embedding(source_input)
            packed = torch.nn.utils.rnn.pack_padded_sequence(source_embedded, source_len, batch_first=True, enforce_sorted=True) #这里enfore_sotred=True要求数据根据词数排序
            source_output, hidden = self.source_gru(packed)
            # source_output=[batch_size, seq_len, 2 * self.hidden_size]，这里的2*self.hidden_size = hidden_dim
            # hidden=[n_layers * 2, batch_size, self.hidden_size]
            source_output, _ = torch.nn.utils.rnn.pad_packed_sequence(source_output, batch_first=True, padding_value=self.pad_index, total_length=len(source_input[0])) #这个会返回output以及压缩后的legnths
            
            batch_size = source_input.shape[0]
            seq_len = source_input.shape[1]
            # 保存slot的预测概率
            slot_outputs = torch.zeros(batch_size, seq_len, self.slot_output_size).to(device)       
                
            aligns = source_output.transpose(0,1) # 为了拿到每个时间步的输出特征，即每个时间步的隐藏向量
            
            output_tokens =[]
                
            # 槽识别
            for t in range(seq_len):
                '''
                此时刻时间步的输出隐向量
                '''
                aligned = aligns[t]# [batch_size, hidden_size]
                    
                # 是否需要计算slot attention
                if self.slot_attention_flag:
                    
                    # [batch_size, hidden_size]
                    slot_context = self.slot_context(aligned, source_output)
                    
                    # [batch_size, hidden_size]，意图上下文向量，利用bigru最后一个时间步的隐状态
                    intent_context = self.intent_context(source_output[:,-1,:], source_output)
                    
                    # gate机制，[batch_size]
                    slot_gate = self.slotGate(slot_context, intent_context)
                    
                    # slot_gate:[batch_size, 1]
                    slot_gate = slot_gate.unsqueeze(1)
                    
                    # slot_context_gate:[batch_size, hidden_dim]
                    slot_context_gate = slot_gate * slot_context
                    
                # 否则，利用每个时间步的隐状态与intent context计算slot gate
                else:
                     # [batch_size, hidden_size]，意图上下文向量，利用bigru最后一个时间步的隐状态
                    intent_context = self.intent_context(source_output[:,-1,:], source_output)
                    
                    # gate机制，[batch_size]
                    slot_gate = self.slotGate(source_output[:,t,:], intent_context)
                    
                     # slot_gate:[batch_size, 1]
                    slot_gate = slot_gate.unsqueeze(1)
                    
                    # slot_context_gate:[batch_size, hidden_dim]
                    slot_context_gate = slot_gate * source_output[:,t,:]
                
                
                
                # 预测槽slot, [batch_size, slot_output_size]
                slot_prediction = self.slot_output(slot_context_gate + source_output[:,t,:])
                slot_outputs[:, t, :] = slot_prediction
                
                
            #意图识别
            intent_outputs = self.intent_output(intent_context + source_output[:,-1,:])

            return slot_outputs, intent_outputs
            
        # 训练阶段
        else:
            # 将输入的source进行编码
            # source_embedded:[batch_size, seq_len, source_emb_dim]
            source_embedded = self.source_embedding(source_input)
            packed = torch.nn.utils.rnn.pack_padded_sequence(source_embedded, source_len, batch_first=True, enforce_sorted=True) #这里enfore_sotred=True要求数据根据词数排序
            source_output, hidden = self.source_gru(packed)
            # source_output=[batch_size, seq_len, 2 * self.hidden_size]，这里的2*self.hidden_size = hidden_dim
            # hidden=[n_layers * 2, batch_size, self.hidden_size]
            source_output, _ = torch.nn.utils.rnn.pad_packed_sequence(source_output, batch_first=True, padding_value=self.pad_index, total_length=len(source_input[0])) #这个会返回output以及压缩后的legnths
            
            batch_size = source_input.shape[0]
            seq_len = source_input.shape[1]
            # 保存slot的预测概率
            slot_outputs = torch.zeros(batch_size, seq_len, self.slot_output_size).to(device)       
                
            aligns = source_output.transpose(0,1) # 为了拿到每个时间步的输出特征，即每个时间步的隐藏向量
                
            # 槽识别
            for t in range(seq_len):
                '''
                此时刻时间步的输出隐向量
                '''
                aligned = aligns[t]# [batch_size, hidden_size]
                    
                # 是否需要计算slot attention
                if self.slot_attention_flag:
                    
                    # [batch_size, hidden_size]
                    slot_context = self.slot_context(aligned, source_output)
                    
                    # [batch_size, hidden_size]，意图上下文向量，利用bigru最后一个时间步的隐状态
                    intent_context = self.intent_context(source_output[:,-1,:], source_output)
                    
                    # gate机制，[batch_size]
                    slot_gate = self.slotGate(slot_context, intent_context)
                    
                    # slot_gate:[batch_size, 1]
                    slot_gate = slot_gate.unsqueeze(1)
                    
                    # slot_context_gate:[batch_size, hidden_dim]
                    slot_context_gate = slot_gate * slot_context
                    
                # 否则，利用每个时间步的隐状态与intent context计算slot gate
                else:
                     # [batch_size, hidden_size]，意图上下文向量，利用bigru最后一个时间步的隐状态
                    intent_context = self.intent_context(source_output[:,-1,:], source_output)
                    
                    # gate机制，[batch_size]
                    slot_gate = self.slotGate(source_output[:,t,:], intent_context)
                    
                     # slot_gate:[batch_size, 1]
                    slot_gate = slot_gate.unsqueeze(1)
                    
                    # slot_context_gate:[batch_size, hidden_dim]
                    slot_context_gate = slot_gate * source_output[:,t,:]
                
                
                
                # 预测槽slot, [batch_size, slot_output_size]
                slot_prediction = self.slot_output(slot_context_gate + source_output[:,t,:])
                slot_outputs[:, t, :] = slot_prediction
                
                
            #意图识别
            intent_outputs = self.intent_output(intent_context + source_output[:,-1,:])

            return slot_outputs, intent_outputs
        

# 构建模型，优化函数，损失函数，学习率衰减函数
def build_model(source, target, label, source_emb_dim, hidden_dim, n_layers, dropout, lr, gamma, weight_decay, seq_len):
    '''
    训练seq2seq model
    input与output的维度是字典的大小。
    encoder与decoder的embedding与dropout可以不同
    网络的层数与hiden/cell状态的size必须相同
    '''
    input_dim = len(source.vocab) # source 词典大小（即词数量）
    output_dim = len(target.vocab) # target 词典大小（即实体类型数量）
    label_dim = len(label.vocab) # label 词典大小（即意图类别数量）
    
    model = BirnnAttentionGate(input_dim, source_emb_dim, hidden_dim, n_layers, dropout, source.vocab.stoi[source.pad_token], output_dim, label_dim, seq_len, False, False).to(device)
   
    model.apply(init_weights)

    # 定义优化函数
    optimizer = optim.Adam(model.parameters(), lr=lr) #, weight_decay=weight_decay)
    #optimizer = torch.optim.SGD(model.parameters(), lr=lr) #, momentum=0.9, nesterov=True)
    # 定义lr衰减
    #scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=gamma)
    
    '''
        当网络的评价指标不在提升的时候，可以通过降低网络的学习率来提高网络性能:
        optimer指的是网络的优化器
        mode (str) ，可选择‘min’或者‘max’，min表示当监控量停止下降的时候，学习率将减小，max表示当监控量停止上升的时候，学习率将减小。默认值为‘min’
        factor 学习率每次降低多少，new_lr = old_lr * factor
        patience=10，容忍网路的性能不提升的次数，高于这个次数就降低学习率
        verbose（bool） - 如果为True，则为每次更新向stdout输出一条消息。 默认值：False
        threshold（float） - 测量新最佳值的阈值，仅关注重大变化。 默认值：1e-4
        cooldown(int)： 冷却时间“，当调整学习率之后，让学习率调整策略冷静一下，让模型再训练一段时间，再重启监测模式。
        min_lr(float or list):学习率下限，可为 float，或者 list，当有多个参数组时，可用 list 进行设置。
        eps(float):学习率衰减的最小值，当学习率变化小于 eps 时，则不调整学习率。
    '''
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', factor=0.1, patience=2, verbose=False)
    # 这里忽略<pad>的损失。
    target_pad_index = target.vocab.stoi[source.pad_token]
    # 定义损失函数(实体识别)
    loss_slot = nn.CrossEntropyLoss(ignore_index=target_pad_index)
    # 定义损失函数(意图识别)
    loss_intent = nn.CrossEntropyLoss()
    
    return model, optimizer, scheduler, loss_slot, loss_intent


# 训练
def train(model, iterator, optimizer, loss_slot, loss_intent, clip):
    '''
    开始训练：
        1.得到source与target句子
        2.上一批batch的计算梯度归0
        3.给模型喂source与target，并得到输出output
        4.由于损失函数只适用于带有1维target和2维的input，我们需要用view进行flatten(在计算损失时，从output与target中忽略了第一列<sos>)
        5.反向传播计算梯度loss.backward()
        6.梯度裁剪，防止梯度爆炸
        7.更新模型参数
        8.损失值求和(返回所有batch的损失的均值)
    '''
    model.train()
    epoch_loss = 0

    for i, batch in enumerate(iterator):
        src, src_lens = batch.source  # src=[batch_size, seq_len]，这里batch.src返回src和src的长度，因为在使用torchtext.Field时设置include_lengths=True
        trg, _ = batch.target  # trg=[batch_size, seq_len]
        label = batch.intent # [batch_size]
        src = src.to(device)
        trg = trg.to(device)
        label = label.to(device)
        
        #slot_outputs=[batch_size, trg_len, trg_vocab_size], intetn_outputs=[batch_size, intent_size]
        slot_outputs, intent_outputs = model(src, src_lens)
        
        # 以下在计算损失时，忽略了每个tensor的第一个元素及<sos>
        output_dim = slot_outputs.shape[-1]
        slot_outputs = slot_outputs[:, 1:, :].reshape(-1, output_dim)  # output=[batch_size * (seq_len - 1), output_dim]
        trg = trg[:, 1:].reshape(-1)  # trg=[batch_size * (seq_len - 1)]
        loss1 = loss_slot(slot_outputs, trg)
        loss2 = loss_intent(intent_outputs, label)
        loss = loss1 + loss2
        
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        optimizer.zero_grad()
        epoch_loss += float(loss.item())
        # print('epoch_loss:{}'.format(float(loss.item())))
    return epoch_loss / len(iterator)

'''
评估
'''
def evaluate(model, iterator, loss_slot, loss_intent):
    model.eval()  # 评估模型，切断dropout与batchnorm
    epoch_loss = 0
    with torch.no_grad():  # 不更新梯度
        for i, batch in enumerate(iterator):
            src, src_len = batch.source  # src=[batch_size, seq_len]
            trg, _ = batch.target  # trg=[batch_size, seq_len]
            label = batch.intent
            src = src.to(device)
            trg = trg.to(device)
            label = label.to(device)
            # output=[batch_size, seq_len, output_dim]
            slot_outputs, intent_outputs = model(src, src_len)

            output_dim = slot_outputs.shape[-1]
            slot_outputs = slot_outputs[:, 1:, :].reshape(-1, output_dim)  # output=[batch_size * (seq_len - 1), output_dim]
            trg = trg[:, 1:].reshape(-1)  # trg=[batch_size * (seq_len - 1)]
            loss1 = loss_slot(slot_outputs, trg)
            loss2 = loss_intent(intent_outputs, label)
            loss = loss1 + loss2
            epoch_loss += float(loss.item())
    return epoch_loss / len(iterator)


def train_model(model, train_iterator, val_iterator, optimizer, scheduler, loss_slot, loss_intent, n_epochs, clip, model_path, writer):
    '''
    开始训练我们的模型：
    1.每一次epoch，都会检查模型是否达到的最佳的validation loss，如果达到了，就更新
    最好的validation loss以及保存模型参数
    2.打印每个epoch的loss以及困惑度。
    '''
    best_valid_loss = float('inf')
    for epoch in range(n_epochs):
        start_time = time.time()
        train_loss = train(model, train_iterator, optimizer, loss_slot, loss_intent, clip)
        writer.add_scalar('loss',train_loss,global_step=epoch+1)
        
        valid_loss = evaluate(model, val_iterator, loss_slot, loss_intent)
        end_time = time.time()
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), model_path)
        scheduler.step(valid_loss)
        print('epoch:{},time-mins:{},time-secs:{}'.format(epoch + 1, epoch_mins, epoch_secs))
        print('train loss:{},train perplexity:{}'.format(train_loss, math.exp(train_loss)))
        print('val loss:{}, val perplexity:{}'.format(valid_loss, math.exp(valid_loss)))
    writer.flush()
    writer.close()

    #每个epoch所花时间
def epoch_time(start_time, end_time):
    run_tim = end_time - start_time
    run_mins = int(run_tim / 60)
    run_secs = int(run_tim-(run_mins * 60))
    return run_mins,run_secs

#对所有模块和子模块进行权重初始化
def init_weights(model):
    for name,param in model.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)


In [12]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(os.getcwd()+'/log', comment='intent_slot')


source_emb_dim = 64
hidden_dim = 128
n_layers = 1
dropout = 0.5
lr = 0.01
gamma = 0.1
weight_decay = 0.1
n_epochs = 10
clip = 5.0
seq_len = 50
model_path = os.path.join(os.getcwd(), "model.h5")


model, optimizer, scheduler, loss_slot, loss_intent = build_model(SOURCE,
                                                                  TARGET,
                                                                  LABEL,
                                                                  source_emb_dim,
                                                                  hidden_dim,
                                                                  n_layers,
                                                                  dropout,
                                                                  lr,
                                                                  gamma,
                                                                  weight_decay,
                                                                  seq_len)

model, optimizer = amp.initialize(model, optimizer, opt_level='O1')

train_model(model,
            train_iter,
            val_iter,
            optimizer,
            scheduler,
            loss_slot, 
            loss_intent,
            n_epochs,
            clip,
            model_path,
            writer)



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 40